In [41]:
!pip install gurobipy

In [43]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [37]:
# Matriz de distância Euclidiana:
matriz_custos = [[0.0, 38.47, 55.95, 18.03, 49.09, 51.48, 38.21, 38.42, 52.39, 14.87],
                [38.47, 0.0, 31.78, 40.8, 45.54, 89.9, 62.51, 7.21, 85.91, 53.23],
                [55.95, 31.78, 0.0, 66.22, 77.28, 101.86, 59.62, 25.18, 108.23, 67.8],
                [18.03, 40.8, 66.22, 0.0, 33.24, 57.31, 55.9, 43.83, 45.28, 26.0],
                [49.09, 45.54, 77.28, 33.24, 0.0, 88.2, 87.01, 52.17, 65.46, 59.24],
                [51.48, 89.9, 101.86, 57.31, 88.2, 0.0, 50.7, 89.19, 37.48, 36.67],
                [38.21, 62.51, 59.62, 55.9, 87.01, 50.7, 0.0, 58.03, 74.33, 34.01],
                [38.42, 7.21, 25.18, 43.83, 52.17, 89.19, 58.03, 0.0, 88.23, 52.7],
                [52.39, 85.91, 108.23, 45.28, 65.46, 37.48, 74.33, 88.23, 0.0, 43.57],
                [14.87, 53.23, 67.8, 26.0, 59.24, 36.67, 34.01, 52.7, 43.57, 0.0]]

In [38]:
n = len(matriz_custos) # Número de clientes
m = 2 # Número de veículos

# Dados de entrada (exemplo)
N = list(range(n))  # 0 = depósito, 1...n = clientes
K = list(range(m))  # veículos

# Dicionario de custos
c = dict()
for k in range(m):
    for i in range(n):
        for j in range(n):
            c[i,j] = matriz_custos[i][j]

# Demanda dos clientes
d = [0, 11, 35, 2, 9, 3, 18, 8, 10, 11]
d_min = min(d)  # demanda mínima

# Capacidade do veículo
Q = 60

100


In [39]:
# Criando do modelo
m = gp.Model("CVRP_DFJ")

# Variáveis
x = m.addVars(N, N, K, vtype=GRB.BINARY, name="x")
# Variáveis u[i] para todos os clientes (não inclui depósito 0)
u = m.addVars(range(1, len(N)), vtype=GRB.CONTINUOUS, lb=d_min, ub=Q, name='u')

# Função objetivo
m.setObjective(gp.quicksum(c[i, j] * x[i, j, k]
                           for i in N for j in N if i != j for k in K), GRB.MINIMIZE)

# Cada veículo sai do depósito uma vez
m.addConstrs(gp.quicksum(x[0, j, k] for j in N if j != 0) == 1 for k in K)
# Cada veículo retorna ao depósito uma vez
m.addConstrs(gp.quicksum(x[i, 0, k] for i in N if i != 0) == 1 for k in K)
# Capacidade dos veículos
m.addConstrs(gp.quicksum(d[i] * x[i, j, k]
                         for i in N if i != 0 for j in N if i != j) <= Q for k in K)
# Cada cliente é visitado uma vez
m.addConstrs(gp.quicksum(x[i, j, k]
                         for k in K for j in N if i != j) == 1 for i in N if i != 0)
# Fluxo de entrada = saída em cada nó (conservação de fluxo)
m.addConstrs(gp.quicksum(x[i, h, k] for i in N if i != h) ==
             gp.quicksum(x[h, j, k] for j in N if j != h) for h in N for k in K)

# Restrições MTZ adaptadas para CVRP
m.addConstrs(
    u[i] - u[j] + Q * x[i, j, k] <= Q - d[j] for k in range(len(K))
                                                for i in range(1, len(N))
                                                    for j in range(1, len(N))
                                                        if i != j)

m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 179 rows, 209 columns and 1152 nonzeros
Model fingerprint: 0xfde90894
Variable types: 9 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 6e+01]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 642.2400000
Presolve removed 2 rows and 20 columns
Presolve time: 0.01s
Presolved: 177 rows, 189 columns, 1114 nonzeros
Variable types: 9 continuous, 180 integer (180 binary)

Root relaxation: objective 3.117700e+02, 53 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [40]:
#Imprime a matriz variavel de decisão
for k in range(len(K)):
    print("k:", k)
    print()
    for i in range(len(N)):
        print(f"{i}: ", end="\t")
        for j in range(len(N)):
            print(round(x[i,j,k].X), end=" ")
        print()
    print()

k: 0

0: 	0 0 1 0 0 0 0 0 0 0 
1: 	1 0 0 0 0 0 0 0 0 0 
2: 	0 0 0 0 0 0 0 1 0 0 
3: 	0 0 0 0 0 0 0 0 0 0 
4: 	0 0 0 0 0 0 0 0 0 0 
5: 	0 0 0 0 0 0 0 0 0 0 
6: 	0 0 0 0 0 0 0 0 0 0 
7: 	0 1 0 0 0 0 0 0 0 0 
8: 	0 0 0 0 0 0 0 0 0 0 
9: 	0 0 0 0 0 0 0 0 0 0 

k: 1

0: 	0 0 0 0 0 0 0 0 0 1 
1: 	0 0 0 0 0 0 0 0 0 0 
2: 	0 0 0 0 0 0 0 0 0 0 
3: 	1 0 0 0 0 0 0 0 0 0 
4: 	0 0 0 1 0 0 0 0 0 0 
5: 	0 0 0 0 0 0 0 0 1 0 
6: 	0 0 0 0 0 1 0 0 0 0 
7: 	0 0 0 0 0 0 0 0 0 0 
8: 	0 0 0 0 1 0 0 0 0 0 
9: 	0 0 0 0 0 0 1 0 0 0 



In [ ]:
# Dada uma cidade j e um veículo k, encontra o próximo nó na rota
def encontrar_prox(x, j, k):
    for i in range(len(N)):
        if i != j and round(x[j, i, k].X) == 1:
            return i
    return None  # Rota encerrada

# Retorna uma sub-rota (lista de nós) para um veículo k, começando do depósito
def escrever_subtour(x, k):
    rota = [0]  # começa no depósito
    next_node = encontrar_prox(x, 0, k)
    while next_node is not None and next_node != 0:
        rota.append(next_node)
        next_node = encontrar_prox(x, next_node, k)
    rota.append(0)  # retorna ao depósito
    return rota

# Retorna uma lista com as rotas de cada veículo
def escrever_solucao(x):
    solucoes = []

    for k in range(len(K)):
        rota = escrever_subtour(x, k)
        # Só adiciona se visitou algum cliente
        if len(rota) > 2:
            solucoes.append((k, rota))
    return solucoes


'''
# Resolve o modelo criando uma restricao a cada sub-rota
qtd_rotas = 0
while qtd_rotas != 1:

    m.optimize()

    solucoes = escrever_solucao(x)
    qtd_rotas = len(solucoes)
    for k, rota in solucoes:
        # Remove os depósitos da rota (se quiser aplicar corte apenas nos clientes)
        rota_sem_deposito = [i for i in rota if i != 0]
        if len(rota_sem_deposito) >= 2:
            m.addConstr(gp.quicksum(x[i, j, k] for i in rota_sem_deposito for j in rota_sem_deposito if i != j) <= len(rota_sem_deposito) - 1)
'''


'\n# Resolve o modelo criando uma restricao a cada sub-rota\nqtd_rotas = 0\nwhile qtd_rotas != 1:\n\n    m.optimize()\n\n    solucoes = escrever_solucao(x)\n    qtd_rotas = len(solucoes)\n    for k, rota in solucoes:\n        # Remove os depósitos da rota (se quiser aplicar corte apenas nos clientes)\n        rota_sem_deposito = [i for i in rota if i != 0]\n        if len(rota_sem_deposito) >= 2:\n            m.addConstr(gp.quicksum(x[i, j, k] for i in rota_sem_deposito for j in rota_sem_deposito if i != j) <= len(rota_sem_deposito) - 1)\n'